In [ ]:
# Jason's portion of TTh Group 6 challenge

import pandas as pd
from pandas.io.json import json_normalize
import requests
import time

from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players

from config import api_key

In [ ]:
# Get complete player list from NBA api
player_list = players.get_players()

# Create a dataframe of all players
player_list_df = pd.DataFrame(player_list)

# Check the df.
player_list_df

In [ ]:
# Use the player IDs we recovered to identify player career stats
# set up query for playercareerstats from NBA api
# Loop through each player


code breaker, do not run this again, it takes over two hours to run as configured.

player_df = pd.DataFrame()
for index, row in player_list_df.iterrows():
    # get player_list_df from df
    
    player_id = row["id"]
    career = playercareerstats.PlayerCareerStats(player_id = player_id)
    response = career.get_data_frames()
    
     # Try to grab the data if available from the API.
    try:
        for player in response:
            player_df = player_df.append(player, sort=True) 
            print(f"ID:{player_id} found! Appending the DataFrame.")
            #Add delay to prevent throttling the API.
            #time.sleep(0.5)
    except:
        print("Player not found")
        pass

In [ ]:
# Save Data to csv
player_df.to_csv("data/player_df.csv")

player_df.columns

In [ ]:
# Merge the player_df with the players list df because the new player_df with the stats doesn't have the player's name
# Using left merge to keep all of the stats data with merge based on the common player ID

player_stats = pd.merge(player_df, player_list_df, left_on="PLAYER_ID", right_on="id", how="left")

#Check the new DF.
player_stats.head()

In [ ]:
# Save Data to csv
player_stats.to_csv("data/player_stats.csv")

In [ ]:
# Create new DF with only 2019 season data

stats_2019 = player_stats.loc[(player_stats["SEASON_ID"] == "2019-20")]

#Check the new DF.
stats_2019.head()

In [ ]:
# Save the 2019 season stats data for each player to csv
stats_2019.to_csv("data/stats_2019.csv")

In [ ]:
# Pull active player data from SportData.io API that encompasses the 2019 season

#Build the base URL
sd_active_players_base_url = "http://api.sportsdata.io/v3/nba/scores/json/Players"

#Building the working url with the API key from config.
sd_active_players_url = sd_active_players_base_url + "?key=" + api_key

# Get data for active players from Sportdata.io

sd_active_players_response = requests.get(sd_active_players_url)
sd_active_players_json = sd_active_players_response.json()

# Normalzie the json from the list of active players from the Sportdata.io
sd_active_players = json_normalize(sd_active_players_json)

# Create the Dataframe
active_players_data_df = pd.DataFrame(active_players_sd)

# Check the DF
active_players_data_df

In [ ]:
# Merge this DataFrame with the stats_2019 DataFrame
# Using inner merge to keep only data common to both the DataFrames 
# linking on the "id" from the NBA and the "NbaDotComPlayerID" from SportsDataIO

combined_stats_2019 = pd.merge(stats_2019, sd_active_players, 
                               left_on="id", right_on="NbaDotComPlayerID", how="inner")

# Check the new DataFrame
combined_stats_2019

In [ ]:
# Drop rows with empty key data starting with "PTS"
nan_value = float("NaN")
combined_stats_2019.replace("", nan_value, inplace=True)

cleaned_2019 = combined_stats_2019.dropna(subset = ["PTS"], inplace=False)

In [ ]:
# Save the 2019 combined stats data for each player to csv
cleaned_2019.to_csv("data/cleaned_2019.csv")

In [ ]:
# Send data to Antonia for further cleaning in nba_cleanup.ipynb